**Neural Chef Assistance**  
**Name: Darren Jer Shien Yee**  
**Student ID: 31237223**

**RNN without Attention**

In [1]:
## Requirements
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [4]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [5]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [9]:
import pandas as pd
df = pd.read_csv('Cooking_Dataset/Cooking_Dataset/train.csv')
print(df.head())  # Display first few rows
print(df.info())  # Summary information about the DataFrame
print(df.describe())  # Summary statistics for numerical columns

                   Title                        Categories     Servings  \
0             cocoa bars                           cookies      24 bars   
1    cocoa bourbon balls         christmas\tcookies\tm. c.   1 servings   
2  cocoa branana muffins          breakfast\tmuffins\tgood  12 servings   
3            cocoa bread                            breads   6 servings   
4    cocoa bread pudding  chocolate\tpuddings\tlow-fat/low   6 servings   

                                         Ingredients  \
0  6 tb butter or margarine - softened\t3/4 c  c ...   
1  1 c  vanilla wafer cookies, finely crushed\t1 ...   
2  1 c  all-purpose flour\t1/2 c  sugar\t2 ts bak...   
3  5 1/2 c  flour\t1    cake compressed yeast\tor...   
4  bread -- to fill dish 2/3\tful\t2 c  skim milk...   

                                              Recipe  
0  cream together butter and powdered sugar . ble...  
1  combine dry ingredients and mix well . add cor...  
2  kellogg 's all-bran 1 . stir together flour 